# CNN on CIFAR-10 Dataset

### Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

### Device Configuartion

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Hyper-parameters

In [3]:
num_epochs = 8
batch_size = 4
learning_rate = 0.001

### Transforms

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

### Dataset & Dataloader

In [5]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### CNN Model

In [8]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 12, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 48, 5)
        self.fc1 = nn.Linear(48 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 48 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
model = ConvNet().to(device)

### Criterion & Optimizer

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Training Loop

In [11]:
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}") as t:
        for i, (images, labels) in enumerate(t):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i + 1) % 100 == 0:
                t.set_postfix(loss=loss.item())

        # for i, (images, labels) in enumerate(train_loader):
        #     images = images.to(device)
        #     labels = labels.to(device)
        #     outputs = model(images)
        #     loss = criterion(outputs, labels)
        #     optimizer.zero_grad()
        #     loss.backward()
        #     optimizer.step()
        #     if (i + 1) % 2000 == 0:
        #         print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{n_total_steps}], Loss: {loss.item():.4f}")

print("Finished Training")

Epoch 8/8: 100%|██████████| 12500/12500 [01:23<00:00, 148.83it/s, loss=0.502] 

Finished Training


### Evaluate

In [13]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]

            n_class_samples[label] += 1
            n_class_correct[label] += (pred == label).item()
    accuracy = 100.0 * n_correct / n_samples
    print(f"Accuracy of the network on the 10000 test images: {accuracy:.4f}")
    for i in range(10):
        class_accuracy = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f"Accuracy of {classes[i]} : {class_accuracy}")

Accuracy of the network on the 10000 test images: 65.5200
Accuracy of plane : 62.4
Accuracy of car : 76.3
Accuracy of bird : 48.6
Accuracy of cat : 42.9
Accuracy of deer : 63.9
Accuracy of dog : 58.6
Accuracy of frog : 71.8
Accuracy of horse : 70.9
Accuracy of ship : 81.3
Accuracy of truck : 78.5
